In [14]:
# Generating Dataset 
import cv2

In [2]:
def generate_dataset():
    face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):# function for Cropping the face from the image
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)# first converting the color image to gray
        faces=face_classifier.detectMultiScale(gray,1.3,5)# To identify the coordinates aroung the face
        if faces is ():
            return None
        for(x,y,w,h) in faces:
            cropped_face=img[y:y+h,x:x+w]
        return cropped_face
    cap=cv2.VideoCapture(0)
    img_id=0
    
    while True:
        ret,frame=cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face=cv2.resize(face_cropped(frame),(200,200))
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            file_name_path="images/"+"A."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)# storing the images into the file
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            
            cv2.imshow("Cropped Faces",face)
            if cv2.waitKey(1)==13 or int(img_id)==1000:# ASCII value of enter button is 13 hence it will exit the infinite loop after pressing enter or will break after storing 1000 images
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is ready!!!")

generate_dataset()#calling the function to generate the dataset

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\vikas\AppData\Local\Temp\ipykernel_20212\4249230928.py:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [15]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob

In [16]:
IMAGE_SIZE=[224,224]

train_path='dataset/train'
valid_path='dataset/test'

vgg = VGG16(weights='imagenet',
            include_top=False,# We are removing the last layer of the enwral network
            input_shape=[224,224,3])

#We don't have to train every layer as each layer is already trained using the imagenet clasification
for layer in vgg.layers:
    layer.trainable=False
    
# The last layer will be folders int he train dataset 
folders=glob('dataset/train/*')
x=Flatten()(vgg.output)
# hence we are flattening the 2d matrix into a 1d matrix
prediction=Dense(len(folders),activation='softmax')(x) #Now we are adding the last layer i.e., the train dataset


# Here we are creating a model object
model=Model(inputs=vgg.input,outputs=prediction)
model.summary() # we can see the model structure

model.compile(
    loss='categorical_crossentropy', # we are classifying the images into multiple names
    optimizer='adam',
    metrics=['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

training_set=train_datagen.flow_from_directory('dataset/train',target_size=(224,224),batch_size=32,class_mode='categorical')

test_set=test_datagen.flow_from_directory('dataset/test',target_size=(224,224),batch_size=32,class_mode='categorical')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [10]:
r=model.fit_generator(training_set,
                     validation_data=test_set,
                     epochs=5,
                     steps_per_epoch=len(training_set),
                     validation_steps=len(test_set))


# import matplotlib.pyplot as plt

# # Loss
# plt.plot(r.history['loss'],label='train loss')
# plt.plot(r.history['val_loss'],label='val loss')
# plt.legend()
# plt.show()

# # Accuracies
# plt.plot(r.history['accuracy'],label='train acc')
# plt.plot(r.history['val_acc'],label='val acc')
# plt.legend()
# plt.show()

from tensorflow.keras.models import save_model
save_model(model, "model.h5")

# import tensorflow as tf
# from keras.models import load_model
# model.save('model1')

C:\Users\vikas\AppData\Local\Temp\ipykernel_6404\1125001928.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r=model.fit_generator(training_set,


Epoch 1/5
11/11 [==============================] - 206s 19s/step - loss: 9.8174e-04 - accuracy: 1.0000 - val_loss: 0.3968 - val_accuracy: 0.8133
Epoch 2/5
11/11 [==============================] - 151s 14s/step - loss: 9.4210e-04 - accuracy: 1.0000 - val_loss: 0.4154 - val_accuracy: 0.7867
Epoch 3/5
11/11 [==============================] - 147s 14s/step - loss: 5.7877e-04 - accuracy: 1.0000 - val_loss: 0.4244 - val_accuracy: 0.7733
Epoch 4/5
11/11 [==============================] - 157s 15s/step - loss: 7.1240e-04 - accuracy: 1.0000 - val_loss: 0.4271 - val_accuracy: 0.7800
Epoch 5/5
11/11 [==============================] - 148s 14s/step - loss: 5.5884e-04 - accuracy: 1.0000 - val_loss: 0.4327 - val_accuracy: 0.7867


In [17]:
from datetime import datetime       
import time
def mark_attendance(name):
    with open('ATTENDANCE.csv','r+') as f:
        current_list=f.readlines()
        
        names=[]
        
        for item in current_list:
            entry=item.split(",")   
            names.append(entry[0])
            
        
        if name not in names:
            cur_time=datetime.now()
            formatted_time=cur_time.strftime("%H:%M:%S")
            f.writelines(f"\n{name},{formatted_time}")

In [18]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.preprocessing import image
import numpy as np
# from keras.models import load_model
# import tensorflow.keras.Models.load_model

from tensorflow.keras.models import load_model
model = load_model('model.h5')

# model = load_model('model1.h5')

In [20]:
# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Prediction
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    #canvas = detect(gray, frame)
    #image, face =face_detector(frame)
    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because model trained with this image size.
        img_array = np.array(im)
                    #model used a 4D tensor, (images x height x width x channel)
                    #changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
                     
        name="None matching"
        
        if(pred[0][0]>0.7):
            name='Elon Musk'
        elif(pred[0][1]>0.7):
            name='Gautum Adani'
        elif(pred[0][2]>0.7):
            name='Vikas Rawat'
#         elif(pred[0][3]>0.7):
#             name='Narendra Modi'
#         elif(pred[0][4]>0.7):
#             name='Vikas Rawat'
        if name=="None matching":
            pass
        mark_attendance(name)
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()

<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\vikas\AppData\Local\Temp\ipykernel_4680\979326619.py:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


1/1 [==============================] - 0s 494ms/step
[[7.2707668e-34 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.5819678e-19]]
1/1 [==============================] - 1s 508ms/step
[[3.2024332e-28 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.9011464e-28]]
1/1 [==============================] - 0s 321ms/step
[[2.5451260e-32 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.3675785e-15]]
1/1 [==============================] - 0s 341ms/step
[[4.6887000e-21 0.0000000e+00 1.0000000e+00 0.0000000e+00 4.5425557e-28]]
1/1 [==============================] - 0s 319ms/step
[[2.9027308e-28 0.0000000e+00 1.0000000e+00 0.0000000e+00 2.4298093e-24]]
1/1 [==============================] - 0s 359ms/step
[[7.7952504e-26 0.0000000e+00 1.0000000e+00 0.0000000e+00 5.9883613e-30]]
1/1 [==============================] - 0s 321ms/step
[[1.5575189e-19 3.5548616e-34 1.0000000e+00 0.0000000e+00 6.2145692e-23]]
1/1 [==============================] - 0s 302ms/step
[[7.3372244e-25 0.0000000e+00 1.0000000e+00 0.0000000e+00 

1/1 [==============================] - 0s 318ms/step
[[0.         0.         0.00308792 0.         0.996912  ]]
1/1 [==============================] - 0s 324ms/step
[[0.0000000e+00 0.0000000e+00 1.4500113e-36 0.0000000e+00 1.0000000e+00]]
1/1 [==============================] - 0s 337ms/step
[[2.2821227e-33 0.0000000e+00 1.0000000e+00 0.0000000e+00 6.4610727e-22]]
1/1 [==============================] - 0s 329ms/step
[[0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.4563467e-22]]
1/1 [==============================] - 0s 338ms/step
[[0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.1088144e-21]]
1/1 [==============================] - 0s 345ms/step
[[6.0098748e-38 0.0000000e+00 9.9997389e-01 0.0000000e+00 2.6088488e-05]]
1/1 [==============================] - 0s 348ms/step
[[9.6777986e-32 0.0000000e+00 2.7019683e-01 0.0000000e+00 7.2980314e-01]]
1/1 [==============================] - 0s 321ms/step
[[5.5292348e-36 0.0000000e+00 1.0000000e+00 0.0000000e+00 4.3257590e-31]]